In [ ]:
import pandas as pd
import numpy as np

### [데이터타입 통합]
kopo_channel_seasonality_new.csv 파일을 불러온 후
selloutData 변수에 담으세요
이후 QTY컬럼→실수(float), 이외컬럼→문자(str)로 변경하세요

In [ ]:
# selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")
selloutData = pd.read_csv("https://raw.githubusercontent.com/hyokwan/python-lecture/refs/heads/master/dataset/kopo_channel_seasonality_new.csv")
selloutData

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0
...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0
124654,A10,PRODUCT56,201630,275.0
124655,A10,PRODUCT61,201630,0.0
124656,A10,PRODUCT12,201630,15021.0


In [ ]:

selloutData.columns
selloutData.REGIONID = selloutData.REGIONID.astype(str)
selloutData.PRODUCT = selloutData.PRODUCT.astype(str)
selloutData.YEARWEEK = selloutData.YEARWEEK.astype(str)
selloutData.QTY = selloutData.QTY.astype(float)
# strColumnList = ["REGIONID","PRODUCT","YEARWEEK"]
# floatColumnList = ["QTY"]

# for i in range(0, len(strColumnList)):
#     eacColumn = strColumnList[i]
#     selloutData[ eacColumn]  = selloutData[ eacColumn ].astype(str)

### [불량 데이터 처리]
kopo_channel_seasonality_new.csv 자료를 담은
selloutData 변수에서
QTY컬럼 음수(반품)인 경우 0, 양수인 경우 기존 QTY 값
유지하는 로직을 적용하여 QTY_NEW 컬럼을 추가하세요

In [ ]:
selloutData['QTY_NEW'] = np.where(selloutData['QTY'] < 0, 0, selloutData['QTY'])

In [ ]:
# selloutData.loc[ selloutData.QTY < 0 ]

### [데이터 통합]
selloutData 자료에서
YEAR, WEEK 컬럼을 생성하고 WEEK 컬럼 값이 52 이하인
데이터만 refinedSelloutData 변수에 저장하세요


In [ ]:
# 연도와 주차 컬럼을 생성한다.
selloutData['YEAR'] = selloutData['YEARWEEK'].astype(str).str[:4]
selloutData['WEEK'] = selloutData['YEARWEEK'].astype(str).str[4:]

In [ ]:
refinedSelloutData = selloutData.loc [ selloutData.WEEK <= "52" ]

In [ ]:
selloutData.shape
refinedSelloutData.shape

(123864, 7)

### [데이터 정렬]
refinedSelloutData 에서 →
지역, 상품, 연주차 컬럼순으로 오름차순 정렬하여
sortedData 변수에 담으세요


In [ ]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]
sortedData = refinedSelloutData.sort_values(by=sortKey,
                                            ascending=True,
                                            ignore_index=True)
sortedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A00,PRODUCT34,201401,661.0,661.0,2014,01
1,A00,PRODUCT34,201402,679.0,679.0,2014,02
2,A00,PRODUCT34,201403,578.0,578.0,2014,03
3,A00,PRODUCT34,201404,532.0,532.0,2014,04
4,A00,PRODUCT34,201405,516.0,516.0,2014,05
...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201648,4152.0,4152.0,2016,48
123860,A77,PRODUCT12,201649,5086.0,5086.0,2016,49
123861,A77,PRODUCT12,201650,5846.0,5846.0,2016,50
123862,A77,PRODUCT12,201651,4933.0,4933.0,2016,51


### [지역, 상품, 연도 별 집계]
sortedData 에서 지역, 상품, 연도 단위
판매량(QTY_NEW) 의 평균 연산 후
groupData 변수에 담으세요

이후 컬럼명을 QTY_MEAN로 변경하세요


In [ ]:
groupKey = ["REGIONID","PRODUCT","YEAR"]
groupData = sortedData.groupby(by=groupKey)["QTY_NEW"].agg(["mean"]).reset_index()
groupData = groupData.rename(columns={"mean":"QTY_MEAN"})

In [ ]:
refinedSelloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,02
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02
3,A60,PRODUCT47,201402,0.0,0.0,2014,02
4,A60,PRODUCT56,201402,23.0,23.0,2014,02
...,...,...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,824.0,2016,30
124654,A10,PRODUCT56,201630,275.0,275.0,2016,30
124655,A10,PRODUCT61,201630,0.0,0.0,2016,30
124656,A10,PRODUCT12,201630,15021.0,15021.0,2016,30


In [ ]:
groupData.loc[(groupData.REGIONID == "A00") & (groupData.PRODUCT == "PRODUCT34") & (groupData.YEAR == "2014")]

,REGIONID,PRODUCT,YEAR,QTY_MEAN
0,A00,PRODUCT34,2014,275.961538


In [ ]:
groupData

,REGIONID,PRODUCT,YEAR,QTY_MEAN
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538
...,...,...,...,...
2377,A77,PRODUCT1,2015,3030.019231
2378,A77,PRODUCT1,2016,3375.326923
2379,A77,PRODUCT12,2014,2035.788462
2380,A77,PRODUCT12,2015,3540.980769


In [ ]:
joinKey = ["REGIONID", "PRODUCT", "YEAR"]
mergedData = pd.merge(left=refinedSelloutData, right=groupData, on = joinKey, how="left")
mergedData = mergedData.sort_values(by=joinKey)
mergedData
groupKey = ["YEAR"]
mergedData.groupby(by=groupKey)["QTY_MEAN"].agg(["mean"])

,mean
YEAR,
2014,6923.721791
2015,9723.427967
2016,10146.172689


In [ ]:
mergedData = mergedData.rename(columns={"mean":"QTY_MEAN"})
mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN,SEASONALITY
205,A00,PRODUCT34,201404,532.0,532.0,2014,04,275.961538,1.927805
298,A00,PRODUCT34,201401,661.0,661.0,2014,01,275.961538,2.395261
1757,A00,PRODUCT34,201402,679.0,679.0,2014,02,275.961538,2.460488
3125,A00,PRODUCT34,201403,578.0,578.0,2014,03,275.961538,2.094495
4313,A00,PRODUCT34,201406,423.0,423.0,2014,06,275.961538,1.532822
...,...,...,...,...,...,...,...,...,...
120622,A77,PRODUCT12,201642,4292.0,4292.0,2016,42,4837.153846,0.887299
121326,A77,PRODUCT12,201630,7120.0,7120.0,2016,30,4837.153846,1.471940
121906,A77,PRODUCT12,201635,3624.0,3624.0,2016,35,4837.153846,0.749201
123672,A77,PRODUCT12,201634,4019.0,4019.0,2016,34,4837.153846,0.830860


In [ ]:
mergedData["SEASONALITY"] = np.where( mergedData.QTY_MEAN != 0,  mergedData.QTY / mergedData.QTY_MEAN, 0)
mergedData.loc[ (mergedData.REGIONID == "A60") &
                 (mergedData.WEEK == "01") &
                 (mergedData.PRODUCT =="PRODUCT4")]
mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN,SEASONALITY
205,A00,PRODUCT34,201404,532.0,532.0,2014,04,275.961538,1.927805
298,A00,PRODUCT34,201401,661.0,661.0,2014,01,275.961538,2.395261
1757,A00,PRODUCT34,201402,679.0,679.0,2014,02,275.961538,2.460488
3125,A00,PRODUCT34,201403,578.0,578.0,2014,03,275.961538,2.094495
4313,A00,PRODUCT34,201406,423.0,423.0,2014,06,275.961538,1.532822
...,...,...,...,...,...,...,...,...,...
120622,A77,PRODUCT12,201642,4292.0,4292.0,2016,42,4837.153846,0.887299
121326,A77,PRODUCT12,201630,7120.0,7120.0,2016,30,4837.153846,1.471940
121906,A77,PRODUCT12,201635,3624.0,3624.0,2016,35,4837.153846,0.749201
123672,A77,PRODUCT12,201634,4019.0,4019.0,2016,34,4837.153846,0.830860


In [ ]:
groupKey = ["REGIONID", "PRODUCT", "WEEK"]
finalResult = mergedData.groupby(by=groupKey)["SEASONALITY"].agg(["mean"]).reset_index()
finalResult.rename(columns = {"mean":"SEASONALITY_MEAN"})

,REGIONID,PRODUCT,WEEK,SEASONALITY_MEAN
0,A00,PRODUCT34,01,1.570782
1,A00,PRODUCT34,02,1.755540
2,A00,PRODUCT34,03,1.319460
3,A00,PRODUCT34,04,1.490298
4,A00,PRODUCT34,05,1.061909
...,...,...,...,...
41283,A77,PRODUCT12,48,1.352712
41284,A77,PRODUCT12,49,1.094083
41285,A77,PRODUCT12,50,1.386116
41286,A77,PRODUCT12,51,1.255192


# Group By  심화

In [ ]:
sortedData["MA5"] =\
  sortedData.QTY_NEW.rolling(window = 5, center=True, min_periods=3).mean()
sortedData["SEASONALITY"] = sortedData.QTY_NEW / sortedData.MA5
sortedData

groupKey = ["REGIONID", "PRODUCT", "WEEK"]
sortedData.groupby(by=groupKey)["SEASONALITY"].agg(["mean"]).reset_index()

,REGIONID,PRODUCT,WEEK,mean
0,A00,PRODUCT34,01,0.767107
1,A00,PRODUCT34,02,0.872595
2,A00,PRODUCT34,03,0.929402
3,A00,PRODUCT34,04,1.161136
4,A00,PRODUCT34,05,0.868462
...,...,...,...,...
41283,A77,PRODUCT12,48,1.087273
41284,A77,PRODUCT12,49,0.908397
41285,A77,PRODUCT12,50,1.063923
41286,A77,PRODUCT12,51,0.875609


In [ ]:
# 그룹바이 생각하면서 적용했을 때

In [ ]:
sortKey = ["REGIONID", "PRODUCT", "YEARWEEK"]

In [ ]:
sortedData = refinedSelloutData.sort_values(by=sortKey, ignore_index=True)

## 이동평균 함수 그룹바이 함수 개념 숙지

In [ ]:
groupKey = ["REGIONID", "PRODUCT"]


In [ ]:
groupData = sortedData.groupby( groupKey )
eachGroup = groupData.get_group( list (groupData.groups)[0] )
# 위에 개념이 조금 힘들면 아래로 생각하면 쉽다 왜? 그룹 한개를 가지고 디버깅하는거기 때문에
# eachGroup = sortedData.loc[ (sqortedData.REGIONID == "A00") &
#                             (sortedData.PRODUCT == "PRODUCT34")]
eachGroup["MA5"] = eachGroup.QTY_NEW.rolling(window=5,center=True, min_periods=1).mean()
eachGroup

<ipython-input-23-9112131abf76>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eachGroup["MA5"] = eachGroup.QTY_NEW.rolling(window=5,center=True, min_periods=1).mean()


,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,MA5
0,A00,PRODUCT34,201401,661.0,661.0,2014,01,639.333333
1,A00,PRODUCT34,201402,679.0,679.0,2014,02,612.500000
2,A00,PRODUCT34,201403,578.0,578.0,2014,03,593.200000
3,A00,PRODUCT34,201404,532.0,532.0,2014,04,545.600000
4,A00,PRODUCT34,201405,516.0,516.0,2014,05,491.200000
...,...,...,...,...,...,...,...,...
151,A00,PRODUCT34,201648,17.0,17.0,2016,48,18.800000
152,A00,PRODUCT34,201649,19.0,19.0,2016,49,30.600000
153,A00,PRODUCT34,201650,35.0,35.0,2016,50,33.200000
154,A00,PRODUCT34,201651,73.0,73.0,2016,51,37.250000


In [ ]:
def rollingMa5(eachGroup):
    eachGroup["MA5"] = eachGroup.QTY_NEW.rolling(window=5, center=True, min_periods=3).mean()
    eachGroup["SEASONALITY"] = np.where(eachGroup.MA5 != 0, eachGroup.QTY_NEW / eachGroup.MA5, 0)
    return eachGroup

groupKey1 = ["REGIONID", "PRODUCT"]
groupData = sortedData.groupby(by=groupKey1).apply(rollingMa5).reset_index(drop=True)

groupKey2 = ["REGIONID", "PRODUCT", "WEEK"]

final = groupData.groupby(by=groupKey2)["SEASONALITY"].agg(["mean"])
final

<ipython-input-27-9386fbcab28d>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  groupData = sortedData.groupby(by=groupKey1).apply(rollingMa5).reset_index(drop=True)


mean
REGIONID PRODUCT   WEEK          
A00      PRODUCT34 01    0.767107
                   02    0.872595
                   03    0.929402
                   04    1.161136
                   05    0.868462
...                           ...
A77      PRODUCT12 48    1.087273
                   49    0.908397
                   50    1.063923
                   51    0.875609
                   52    0.968998

[41288 rows x 1 columns]

In [ ]:
def rollingMa5(eachGroup):
    # 디버깅코드
    # groupData = sortedData.groupby( groupKey )
    # eachGroup = groupData.get_group( list (groupData.groups)[0] )
    # 위에 개념이 조금 힘들면 아래로 생각하면 쉽다 왜? 그룹 한개를 가지고 디버깅하는거기 때문에
    # eachGroup = sortedData.loc[ (sqortedData.REGIONID == "A00") &
    #                             (sortedData.PRODUCT == "PRODUCT34")]
    eachGroup["MA5"] = eachGroup.QTY_NEW.rolling(window=5,center=True, min_periods=3).mean()
    return eachGroup
groupData = sortedData.groupby(groupKey).apply(rollingMa5)
final = groupData.reset_index(drop=True)
groupkey = ["REGIONID", "PRODUCT", "WEEK"]
final["SEASONALITY"] = np.where(final.MA5 !=0, final.QTY_NEW / final.MA5, 0)
final2 = eachGroup.groupby(by=groupKey)["SEASONALITY"].agg("mean").reset_index(drop=True)
final2

<ipython-input-92-6090ab8b6e7c>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  groupData = sortedData.groupby(groupKey).apply(rollingMa5)


,SEASONALITY
0,0.983166
